<a href="https://colab.research.google.com/github/BatoolMM/fastai/blob/master/chapter__7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastbook
!pip install fastcore==1.0.9
!pip install utils
!pip install wandb
!pip install nbdev

In [2]:
from utils import *
from fastbook import *
from fastai.vision.all import *
from fastai.vision.widgets import ImageClassifierCleaner
import wandb
from fastai.callback.wandb import WandbCallback
import matplotlib.pyplot as plt

In [3]:
path = untar_data(URLs.IMAGENETTE)

In [4]:
path = path/'val'
path.ls()

(#10) [Path('/root/.fastai/data/imagenette2/val/n03425413'),Path('/root/.fastai/data/imagenette2/val/n03394916'),Path('/root/.fastai/data/imagenette2/val/n01440764'),Path('/root/.fastai/data/imagenette2/val/n03000684'),Path('/root/.fastai/data/imagenette2/val/n02979186'),Path('/root/.fastai/data/imagenette2/val/n02102040'),Path('/root/.fastai/data/imagenette2/val/n03028079'),Path('/root/.fastai/data/imagenette2/val/n03888257'),Path('/root/.fastai/data/imagenette2/val/n03417042'),Path('/root/.fastai/data/imagenette2/val/n03445777')]

In [5]:
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                   get_items = get_image_files,
                   get_y = parent_label,
                   item_tfms = Resize(460),
                   batch_tfms = aug_transforms(size=224, min_scale=0.75))


In [6]:
dls = dblock.dataloaders(path, bs=64)

In [7]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics = accuracy)
learn.fit_one_cycle(5,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.588726,3.395481,0.247134,00:49
1,1.912873,1.785894,0.433121,00:50
2,1.549041,1.777100,0.484076,00:52
3,1.290151,1.212406,0.600000,00:51
4,1.089523,1.086031,0.638217,00:52


In [8]:
x, y = dls.one_batch()

In [9]:
x.mean(dim=[0,2,3])

TensorImage([0.4543, 0.4373, 0.4089], device='cuda:0')

In [10]:
x.std(dim=[0,2,3])

TensorImage([0.2742, 0.2710, 0.2909], device='cuda:0')

In [11]:
def get_dls(bs, size):
  dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                     get_items = get_image_files,
                     get_y = parent_label,
                     item_tfms = Resize(460),
                     batch_tfms = [*aug_transforms(size=size, min_scale=0.75),
                                   Normalize.from_stats(*imagenet_stats)])
  return dblock.dataloaders(path, bs=bs)

In [12]:
dls = get_dls(64, 224)

In [13]:
x, y = dls.one_batch()

In [14]:
x.mean(dim = [0,2,3]), x.std(dim =[0,2,3])

(TensorImage([-0.0517,  0.0184,  0.1575], device='cuda:0'),
 TensorImage([1.2341, 1.2506, 1.3392], device='cuda:0'))

In [15]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.545964,5.035451,0.252229,00:52
1,1.914937,1.752107,0.425478,00:51
2,1.550101,1.457210,0.552866,00:52
3,1.296233,1.138097,0.647134,00:52
4,1.104339,1.026049,0.708280,00:51


In [16]:
dls = get_dls(128,128)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.116194,1.928121,0.476433,00:39
1,1.221076,2.087733,0.412739,00:39
2,1.150762,1.235947,0.619108,00:39
3,1.043803,1.080472,0.662420,00:39
4,0.936024,0.835646,0.743949,00:39


In [17]:
learn.dls = get_dls(64, 224)
learn.fine_tune(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.860326,1.263772,0.569427,00:52


epoch,train_loss,valid_loss,accuracy,time
0,0.761478,0.855474,0.703185,00:52
1,0.747320,0.809742,0.746497,00:52
2,0.716942,0.662412,0.793631,00:52
3,0.659202,0.636972,0.803822,00:52
4,0.601558,0.599353,0.812739,00:52
